<a href="https://colab.research.google.com/github/TimothyJan/RNN_Text_Generation/blob/main/Project_4_RNN_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spring 2022 - CPSC 585-section-13883
Project 4 - Recurrent Neural Networks Text Generation
Sean Javiya
Timothy Jan
Timothy Kheang

In Project 4, we use an RNN-based language model to generate text for a creative application.

Goals for this project are:
<ul>
  <li>Reading about how RNNs can be used to generate text, and examining several different applications after training on different text corpora.</li>
  <li>Using a multi-layer RNN to train and sample from a character-level language model.</li>
  <li>Adapting and reusing published model code.</li>
  <li>Having some fun.</li>
</ul>

In [1]:
import tensorflow as tf
import numpy as np
import time
import urllib.request, json 

Download the Elder Scroll's dataset

In [23]:
# imported the elder scrolls json file
with urllib.request.urlopen("https://raw.githubusercontent.com/hmi-utwente/video-game-text-corpora/master/The%20Elder%20Scrolls/data/imperial_library_20200626.json") as url:
    data_dict = json.loads(url.read().decode())

Read the Data

In [24]:
for key in data_dict:
  text += data_dict[key]['description'] +' ' + data_dict[key]['title'] + ' ' + data_dict[key]['text']
text[:100]

"The  last words of a world-renowned archaeologist. A Dying Man's Last Words It's been many days sinc"

In [22]:
# unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

118 unique characters


Process the text

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)